In [1]:
raw_text = "I made myself a snowball As perfect as could be.\n I thought I'd keep it as a pet And let it sleep with me.\n I made it some pajamas And a pillow for its head.\n Then last night it ran away, But first it wet the bed."

In [2]:
tokens = raw_text.split()

In [3]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([raw_text])

In [5]:
encoded = tokenizer.texts_to_sequences([raw_text])[0]

In [6]:
X = []
y = []

for i in range(len(encoded)-1):
    X.append(encoded[i])
    y.append(encoded[i+1])

In [7]:
import numpy as np

sequences = []
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)
print( ' Total Sequences: %d ' % len(sequences))
# split into X and y elements
sequences = np.array(sequences)
Xs, ys = sequences[:,0],sequences[:,1]

 Total Sequences: 46 


In [8]:
from keras.utils import to_categorical

y = to_categorical(y, num_classes= len(tokenizer.word_index) + 1)

In [9]:
import numpy as np

X, y = np.asarray(X), np.asarray(y)

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [15]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+ 1, 10, input_length = 1))
model.add(LSTM(50))
model.add(Dense(len(tokenizer.word_index) + 1, activation = 'softmax'))

In [16]:
model.compile(loss= 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             360       
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 36)                1836      
Total params: 14,396
Trainable params: 14,396
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X, y, epochs = 100)

Epoch 1/100
46/46 [==============================] - 0s 199us/step - loss: 0.4610 - acc: 0.7609
Epoch 2/100
46/46 [==============================] - 0s 223us/step - loss: 0.4608 - acc: 0.7609
Epoch 3/100
46/46 [==============================] - 0s 194us/step - loss: 0.4606 - acc: 0.7609
Epoch 4/100
46/46 [==============================] - 0s 220us/step - loss: 0.4601 - acc: 0.7609
Epoch 5/100
46/46 [==============================] - 0s 241us/step - loss: 0.4601 - acc: 0.7609
Epoch 6/100
46/46 [==============================] - 0s 219us/step - loss: 0.4595 - acc: 0.7609
Epoch 7/100
46/46 [==============================] - 0s 233us/step - loss: 0.4595 - acc: 0.7609
Epoch 8/100
46/46 [==============================] - 0s 221us/step - loss: 0.4592 - acc: 0.7609
Epoch 9/100
46/46 [==============================] - 0s 327us/step - loss: 0.4589 - acc: 0.7609
Epoch 10/100
46/46 [==============================] - 0s 245us/step - loss: 0.4590 - acc: 0.7609
Epoch 11/100
46/46 [===================

46/46 [==============================] - 0s 238us/step - loss: 0.4466 - acc: 0.7609
Epoch 86/100
46/46 [==============================] - 0s 339us/step - loss: 0.4465 - acc: 0.7609
Epoch 87/100
46/46 [==============================] - 0s 260us/step - loss: 0.4463 - acc: 0.7609
Epoch 88/100
46/46 [==============================] - 0s 317us/step - loss: 0.4462 - acc: 0.7609
Epoch 89/100
46/46 [==============================] - 0s 277us/step - loss: 0.4459 - acc: 0.7609
Epoch 90/100
46/46 [==============================] - 0s 278us/step - loss: 0.4462 - acc: 0.7609
Epoch 91/100
46/46 [==============================] - 0s 283us/step - loss: 0.4458 - acc: 0.7609
Epoch 92/100
46/46 [==============================] - 0s 178us/step - loss: 0.4461 - acc: 0.7609
Epoch 93/100
46/46 [==============================] - 0s 256us/step - loss: 0.4459 - acc: 0.7609
Epoch 94/100
46/46 [==============================] - 0s 311us/step - loss: 0.4455 - acc: 0.7609
Epoch 95/100
46/46 [=======================

In [23]:
def generate_seq(model, tokenizer, seed_text, n_words):
    in_text, result = seed_text, seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = np.array(encoded)
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

In [30]:
generate_seq(model, tokenizer, 'I', 10)

'I made myself a pillow for its head then last night'

In [31]:
generate_seq(model, tokenizer, 'pajamas', 4)

'pajamas and let it wet'